## Deliverable 2. Create a Customer Travel Destinations Map.

In [41]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [16]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.6001,-73.9662,72.30,69,100,5.84,overcast clouds
1,1,Geraldton,AU,-28.7667,114.6000,66.13,77,90,6.91,overcast clouds
2,2,Esperance,AU,-33.8667,121.9000,58.84,63,98,20.33,overcast clouds
3,3,Georgetown,MY,5.4112,100.3354,87.73,79,20,1.01,few clouds
4,4,Pangnirtung,CA,66.1451,-65.7125,47.59,90,100,13.02,moderate rain


In [17]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 95


In [18]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.6001,-73.9662,72.30,69,100,5.84,overcast clouds
1,1,Geraldton,AU,-28.7667,114.6000,66.13,77,90,6.91,overcast clouds
3,3,Georgetown,MY,5.4112,100.3354,87.73,79,20,1.01,few clouds
9,9,Atuona,PF,-9.8000,-139.0333,78.42,70,15,17.85,few clouds
10,10,Thompson,CA,55.7435,-97.8558,60.96,94,90,5.75,overcast clouds
11,11,Great Yarmouth,GB,52.6083,1.7305,63.45,95,100,5.03,overcast clouds
12,12,Kapaa,US,22.0752,-159.3190,84.52,81,90,3.00,light rain
13,13,Seoul,KR,37.5683,126.9778,93.61,44,0,6.91,clear sky
14,14,Rikitea,PF,-23.1203,-134.9692,69.49,56,0,0.18,clear sky
15,15,Langsa,ID,4.4683,97.9683,87.71,61,73,3.83,broken clouds


In [19]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                480
City                   480
Country                476
Lat                    480
Lng                    480
Max Temp               480
Humidity               480
Cloudiness             480
Wind Speed             480
Current Description    480
dtype: int64

In [20]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.copy()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.6001,-73.9662,72.30,69,100,5.84,overcast clouds
1,1,Geraldton,AU,-28.7667,114.6000,66.13,77,90,6.91,overcast clouds
3,3,Georgetown,MY,5.4112,100.3354,87.73,79,20,1.01,few clouds
9,9,Atuona,PF,-9.8000,-139.0333,78.42,70,15,17.85,few clouds
10,10,Thompson,CA,55.7435,-97.8558,60.96,94,90,5.75,overcast clouds


In [21]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Albany,US,72.30,overcast clouds,42.6001,-73.9662,
1,Geraldton,AU,66.13,overcast clouds,-28.7667,114.6000,
3,Georgetown,MY,87.73,few clouds,5.4112,100.3354,
9,Atuona,PF,78.42,few clouds,-9.8000,-139.0333,
10,Thompson,CA,60.96,overcast clouds,55.7435,-97.8558,
11,Great Yarmouth,GB,63.45,overcast clouds,52.6083,1.7305,
12,Kapaa,US,84.52,light rain,22.0752,-159.3190,
13,Seoul,KR,93.61,clear sky,37.5683,126.9778,
14,Rikitea,PF,69.49,clear sky,-23.1203,-134.9692,
15,Langsa,ID,87.71,broken clouds,4.4683,97.9683,


In [26]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    Lat = row["Lat"]
    Lng = row["Lng"]
    params["location"] = f"{Lat},{Lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    print(f"Retrieving Results for Index {index}.")
    response = requests.get(base_url, params=params).json()
    results = response["results"]
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result...skipping.")
        
    print("----------")
        

Retrieving Results for Index 0.
Missing field/result...skipping.
----------
Retrieving Results for Index 1.
----------
Retrieving Results for Index 3.
----------
Retrieving Results for Index 9.
----------
Retrieving Results for Index 10.
----------
Retrieving Results for Index 11.
----------
Retrieving Results for Index 12.
----------
Retrieving Results for Index 13.
----------
Retrieving Results for Index 14.
----------
Retrieving Results for Index 15.
----------
Retrieving Results for Index 16.
Missing field/result...skipping.
----------
Retrieving Results for Index 17.
----------
Retrieving Results for Index 18.
----------
Retrieving Results for Index 19.
----------
Retrieving Results for Index 20.
----------
Retrieving Results for Index 22.
----------
Retrieving Results for Index 25.
Missing field/result...skipping.
----------
Retrieving Results for Index 26.
----------
Retrieving Results for Index 27.
----------
Retrieving Results for Index 28.
----------
Retrieving Results for In

----------
Retrieving Results for Index 264.
----------
Retrieving Results for Index 265.
----------
Retrieving Results for Index 266.
----------
Retrieving Results for Index 268.
----------
Retrieving Results for Index 269.
----------
Retrieving Results for Index 272.
----------
Retrieving Results for Index 273.
----------
Retrieving Results for Index 275.
----------
Retrieving Results for Index 276.
----------
Retrieving Results for Index 277.
----------
Retrieving Results for Index 280.
----------
Retrieving Results for Index 281.
----------
Retrieving Results for Index 282.
----------
Retrieving Results for Index 283.
----------
Retrieving Results for Index 284.
----------
Retrieving Results for Index 285.
----------
Retrieving Results for Index 286.
Missing field/result...skipping.
----------
Retrieving Results for Index 288.
----------
Retrieving Results for Index 290.
----------
Retrieving Results for Index 291.
----------
Retrieving Results for Index 293.
----------
Retrieving 

----------
Retrieving Results for Index 497.
----------
Retrieving Results for Index 499.
----------
Retrieving Results for Index 500.
Missing field/result...skipping.
----------
Retrieving Results for Index 502.
----------
Retrieving Results for Index 503.
----------
Retrieving Results for Index 505.
Missing field/result...skipping.
----------
Retrieving Results for Index 508.
----------
Retrieving Results for Index 509.
----------
Retrieving Results for Index 510.
----------
Retrieving Results for Index 511.
----------
Retrieving Results for Index 512.
----------
Retrieving Results for Index 513.
----------
Retrieving Results for Index 515.
----------
Retrieving Results for Index 516.
----------
Retrieving Results for Index 517.
----------
Retrieving Results for Index 519.
----------
Retrieving Results for Index 520.
----------
Retrieving Results for Index 521.
----------
Retrieving Results for Index 522.
----------
Retrieving Results for Index 523.
----------
Retrieving Results for 

In [43]:
hotel_df["Hotel Name"] = hotel_df["Hotel Name"].replace(r'^\s*$', np.nan, regex=True)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Albany,US,72.30,overcast clouds,42.6001,-73.9662,NaN
1,Geraldton,AU,66.13,overcast clouds,-28.7667,114.6000,Broadwater Mariner Resort
3,Georgetown,MY,87.73,few clouds,5.4112,100.3354,Cititel Penang
9,Atuona,PF,78.42,few clouds,-9.8000,-139.0333,Villa Enata
10,Thompson,CA,60.96,overcast clouds,55.7435,-97.8558,Burntwood Hotel
...,...,...,...,...,...,...,...
661,Rodnikovskaya,RU,69.22,overcast clouds,44.7644,40.6656,NaN
662,Paso De Los Toros,UY,65.16,overcast clouds,-32.8167,-56.5167,Complejo Isabelino
663,Jardim,BR,69.35,clear sky,-21.4803,-56.1381,Estancia Hotel - Jardim MS
664,Port Augusta,AU,60.94,clear sky,-32.5000,137.7667,Majestic Oasis Apartments


In [49]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(how="any")

In [50]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Geraldton,AU,66.13,overcast clouds,-28.7667,114.6000,Broadwater Mariner Resort
3,Georgetown,MY,87.73,few clouds,5.4112,100.3354,Cititel Penang
9,Atuona,PF,78.42,few clouds,-9.8000,-139.0333,Villa Enata
10,Thompson,CA,60.96,overcast clouds,55.7435,-97.8558,Burntwood Hotel
11,Great Yarmouth,GB,63.45,overcast clouds,52.6083,1.7305,Comfort Hotel Great Yarmouth
...,...,...,...,...,...,...,...
660,Long Xuyen,VN,78.75,overcast clouds,10.3833,105.4167,Nhà Nghỉ Sài Gòn Ngày Đêm
662,Paso De Los Toros,UY,65.16,overcast clouds,-32.8167,-56.5167,Complejo Isabelino
663,Jardim,BR,69.35,clear sky,-21.4803,-56.1381,Estancia Hotel - Jardim MS
664,Port Augusta,AU,60.94,clear sky,-32.5000,137.7667,Majestic Oasis Apartments


In [52]:
# 8a. Create the output File (CSV)
output_data_file = "weatherpy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [57]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<d1>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</d1>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [58]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))